In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

### Import necessary packages

In [3]:
#Essentials
import numpy as np
import pandas as pd
import pickle
import re
import datetime as dt
import time

# For scraping
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import os

#SQL related - NEED TO DECIDE WHICH ONE I'LL BE USING AND DELETE THE REST
import psycopg2
import sqlite3
from sqlalchemy import create_engine
import pandas.io.sql as pd_sql

#Visualization
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns


%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 25, 6
rcParams.update({'font.size': 14})

## Gather data

### Helper functions

In [73]:
def is_for_sale_active(url):
    """Checks if 'For Sale' tab is selected, to make sure we're not looking at rentals
    Returns a Boolean (True for For Sale is active, False - if it isn't active)"""
    
def get_todays_ads():
    """Retrieves only adverts posted today"""
    
#Get urls for all the pages and put them into a list
def get_url_list():
    """Records the current url and goes through the website, clicking Next as many times as there are pages.
    Returns a list of urls to be used in the get_htmls function."""
    #Creates a list of urls for all pages
    url_list = []
    
    #Reads the url of the page the driver is currently in and adds it into the list
    current_page_url = driver.current_url
    url_list.append(current_page_url)
    
    curr_page_num = 0
    count_exceptions = 0
    while True:
        try:
            curr_page_num += 1
            #Find "Next" button and click it
            find_next = driver.find_element_by_xpath("//a[@rel='next']")
            find_next.click()
            current_page_url = driver.current_url
            url_list.append(current_page_url)
            time.sleep(10)
        except:
            #Count the exceptions
            count_exceptions += 1
            curr_page_num -= 1 #since we are coming back to the same page
            #If the there haven't been 3 exceptions yet, sleep for a bit and then continue
            if count_exceptions < 3:
                time.sleep(10)
            else:
                #If "Next" button isn't there anymore or an error occurs, return the list
                #driver.close()
                print(f'Last page number was {curr_page_num}.')
                return url_list
    print(f'Last page number was {curr_page_num}.')
    #driver.close()
    return url_list

def scrape_data(html):
    """Reads in the data about each property from a given html file.
    Returns a dictionary (WILL HAVE TO CHANGE THIS IN THE FUTURE)"""
    

#Get all the html files for each property ad and put it into a list
def get_htmls(html_list):
    """Takes current html list.
    Returns updated html list with all the property ad htmls available on the page."""
    
    #Find all property add htmls
    elems = driver.find_elements_by_xpath("//a[@href]")
    for elem in elems:
        html = elem.get_attribute("href")
        html_list.append(html)        
    return html_list


def get_each_feature(highlighted_features):
    """Takes a string with all highlighted features and puts them into a dictionary.
    Returns the dictionary"""
    feature_list = highlighted_features.split('\n')[1:]
    counter = 0
    feat_dict = {}
    
    for feat in feature_list[::2]:
        feat_index = feature_list.index(feat)
        feat_dict[feat] = feature_list[feat_index+1]
    return feat_dict

In [5]:
# At the moment only getting the data from Funda, could have a list of websites in the future
website = 'https://www.funda.nl/en/'

chromedriver = "/Applications/chromedriver"           #  path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [63]:
# Go to the website
driver = webdriver.Chrome(chromedriver)
driver.get(website)

In [65]:


# Now apply desired filters (e.g. set location to Amsterdam)
# try:
filter_loc = driver.find_element_by_name('filter_location')
filter_loc.send_keys('Amsterdam')

# Wait for the first dropdown option to appear and select the first option
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".autocomplete-list")))
filter_loc.send_keys(Keys.ARROW_DOWN)
filter_loc.send_keys(Keys.ENTER)

filter_radius = Select(driver.find_element_by_id('Straal'))
filter_radius.select_by_value('0')

filter_price_from = Select(driver.find_element_by_name('filter_KoopprijsVan'))
filter_price_from.select_by_value('200000')

filter_price_upto = Select(driver.find_element_by_name('filter_KoopprijsTot'))
filter_price_upto.select_by_value('400000')

find_search = driver.find_element_by_xpath("//button[contains(text(), 'Search')]")
find_search.click()

# Now that we applied basic filters, let's narrow the search down more

# except:
#     driver.find_element_by_class_name('recaptcha-checkbox-border').click()
# except:
#     WebElement iFrame_checkbox = driver.findElement(By.xpath("xpath_of_reCaptcha_checkbox"));
#     iFrame_checkbox.click();


# car_type_form.send_keys(Keys.RETURN)

In [91]:
# url_list = get_url_list()
# with open('./Cellar/pages_url_list.pkl', 'wb') as url_pickle:
#     pickle.dump(url_list, url_pickle)

In [96]:
with open('./Cellar/pages_url_list.pkl', 'rb') as url_pickle:
    url_list = pickle.load(url_pickle)

print(f'Imported {len(url_list)} URLs.')

Imported 77 URLs.


In [32]:
count = 0

html_list = []

elems = driver.find_elements_by_xpath("//a[@href]")
for elem in elems:
    html = elem.get_attribute("href")
    if (bool(re.search(r'appartement-\d+', html)) or bool(re.search('huis-\d+', html))) and html not in html_list: 
        html_list.append(html)

In [76]:
test_case = html_list[0]

# To open the property ad
driver.get(test_case)

title = driver.find_element_by_class_name('object-header__title').text
address = driver.find_element_by_class_name('object-header__subtitle').text
price = driver.find_element_by_class_name('object-header__price').text
highlighted_features = driver.find_element_by_class_name('kenmerken-highlighted').text
highlighted_features = get_each_feature(highlighted_features)

# Click to see all the features available
time.sleep(5)
driver.find_element_by_class_name('object-kenmerken-open-button').click()

all_features = driver.find_element_by_class_name('object-kenmerken-body').text



In [77]:
driver.find_element_by_class_name('object-kenmerken-body').text



'Transfer of ownership\nAsking price\n€ 325,000 k.k.\nAsking price per m²\n€ 7,927\nListed since\n2 weeks\nStatus\nAvailable\nAcceptance\nAvailable in consultation\nVVE (Owners Association) contribution\n€ 70 per month\nConstruction\nType apartment\nUpstairs apartment (apartment)\nBuilding type\nResale property\nYear of construction\n1925\nSpecific\nListed building (national monument)\nSurface areas and volume\nAreas\nLiving area\n41 m²\nExterior space attached to the building\n2 m²\nExternal storage space\n7 m²\nVolume in cubic meters\n128 m³\nLayout\nNumber of rooms\n2 rooms (1 bedroom)\nNumber of bath rooms\n1 bathroom\nBathroom facilities\nShower and toilet\nNumber of residential layers (stories)\n1 residential layer (story)\nLocated at\n3rd level of residential structure\nFacilities\nMechanical ventilation and TV via cable\nEnergy\nEnergy label\nA 0.89 What does this mean?\nInsulation\nInsulated walls, floor insulation and double glazing\nHeating\nCH boiler\nHot water\nCH boiler\n

In [78]:
driver.close()